# The Happiness Project - Data Exploration and Cleaning



In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import gmaps
import requests
import json

import statsmodels
import statsmodels.api as sm
from scipy.stats import linregress

### 2021 Happiness Data

In [7]:
# Import World Happiness Report Data 2021
raw_happiness_df = pd.read_csv("Data/world-happiness-report-2021.csv")

# Using DataFrame.insert() to add a column
raw_happiness_df.insert(2, "Year", "2021")

# Rename columns
raw_happiness_df = raw_happiness_df.rename(columns={'Country name': 'Country', 
                                                'Regional indicator': 'Region',
                                                'Ladder score': 'Happiness Score',
                                                'Social support': 'Social Support',                                                    
                                                'Logged GDP per capita': 'GDP per Capita',
                                                'Healthy life expectancy': 'Life Expectancy',
                                                'Freedom to make life choices': 'Freedom',
                                                'Perceptions of corruption': 'Corruption'})

# Drop columns not needed
happiness_df = raw_happiness_df.drop(columns=['Standard error of ladder score', 
                                              'upperwhisker', 
                                              'lowerwhisker',
                                              'Ladder score in Dystopia',
                                              'Explained by: Log GDP per capita',
                                              'Explained by: Social support',
                                              'Explained by: Healthy life expectancy',
                                              'Explained by: Freedom to make life choices',
                                              'Explained by: Generosity',
                                              'Explained by: Perceptions of corruption',
                                              'Dystopia + residual'])

# Write to csv and show preview of DataFrame
happiness_df.to_csv("Data/Final Dataframes/happiness_df.csv", index=False)
happiness_df.head(20)

,Country,Region,Year,Happiness Score,GDP per Capita,Social Support,Life Expectancy,Freedom,Generosity,Corruption
0,Finland,Western Europe,2021,7.842,10.775,0.954,72.000,0.949,-0.098,0.186
1,Denmark,Western Europe,2021,7.620,10.933,0.954,72.700,0.946,0.030,0.179
2,Switzerland,Western Europe,2021,7.571,11.117,0.942,74.400,0.919,0.025,0.292
3,Iceland,Western Europe,2021,7.554,10.878,0.983,73.000,0.955,0.160,0.673
4,Netherlands,Western Europe,2021,7.464,10.932,0.942,72.400,0.913,0.175,0.338
5,Norway,Western Europe,2021,7.392,11.053,0.954,73.300,0.960,0.093,0.270
6,Sweden,Western Europe,2021,7.363,10.867,0.934,72.700,0.945,0.086,0.237
7,Luxembourg,Western Europe,2021,7.324,11.647,0.908,72.600,0.907,-0.034,0.386
8,New Zealand,North America and ANZ,2021,7.277,10.643,0.948,73.400,0.929,0.134,0.242
9,Austria,Western Europe,2021,7.268,10.906,0.934,73.300,0.908,0.042,0.481


### Historical Happiness Data

In [8]:
# Import World Happiness Report Data 2005-2020
raw_happiness_time = pd.read_csv("Data/world-happiness-report.csv")

# Using DataFrame.insert() to add a column
raw_happiness_time.insert(2, "Region", "")

# Rename columns
raw_happiness_time = raw_happiness_time.rename(columns={'Country name': 'Country',
                                                'year' : 'Year',
                                                'Life Ladder': 'Happiness Score',
                                                'Social support': 'Social Support',  
                                                'Healthy life expectancy at birth': 'Life Expectancy',
                                                'Log GDP per capita': 'GDP per Capita',
                                                'Healthy life expectancy': 'Life Expectancy',
                                                'Freedom to make life choices': 'Freedom',
                                                'Perceptions of corruption': 'Corruption'})

# Drop columns not needed
happiness_time = raw_happiness_time.drop(columns=['Positive affect', 
                                                   'Negative affect' 
                                                  ])
happiness_time

,Country,Year,Region,Happiness Score,GDP per Capita,Social Support,Life Expectancy,Freedom,Generosity,Corruption
0,Afghanistan,2008,,3.724,7.370,0.451,50.80,0.718,0.168,0.882
1,Afghanistan,2009,,4.402,7.540,0.552,51.20,0.679,0.190,0.850
2,Afghanistan,2010,,4.758,7.647,0.539,51.60,0.600,0.121,0.707
3,Afghanistan,2011,,3.832,7.620,0.521,51.92,0.496,0.162,0.731
4,Afghanistan,2012,,3.783,7.705,0.521,52.24,0.531,0.236,0.776
...,...,...,...,...,...,...,...,...,...,...
1944,Zimbabwe,2016,,3.735,7.984,0.768,54.40,0.733,-0.095,0.724
1945,Zimbabwe,2017,,3.638,8.016,0.754,55.00,0.753,-0.098,0.751
1946,Zimbabwe,2018,,3.616,8.049,0.775,55.60,0.763,-0.068,0.844
1947,Zimbabwe,2019,,2.694,7.950,0.759,56.20,0.632,-0.064,0.831


In [9]:
# Add data from 2021 to our 2005-2020 DataFrame 
happiness_time_final = pd.concat([happiness_time, happiness_df])

# Convert Year column from string to int
happiness_time_final['Year'] = happiness_time_final['Year'].astype(int)

#  Sort DataFrame by Country and Year for more clarity
happiness_time_final = happiness_time_final.sort_values(['Country', 'Year']).reset_index()

# Add region names
for index, row in happiness_time_final.iterrows():
    
    country =  row['Country']
    if country in happiness_df['Country'].unique():
        row_index = happiness_df.index[happiness_df['Country'] == country][0]
        region = happiness_df.iloc[row_index, 1]
        happiness_time_final.iloc[index, 3] = region

# # # Write to csv and show dataframe preview
happiness_time_final = happiness_time_final.drop("index", axis=1)
happiness_time_final.to_csv("Data/Final Dataframes/happiness_time_final.csv", index=False)
happiness_time_final

,Country,Year,Region,Happiness Score,GDP per Capita,Social Support,Life Expectancy,Freedom,Generosity,Corruption
0,Afghanistan,2008,South Asia,3.724,7.370,0.451,50.800,0.718,0.168,0.882
1,Afghanistan,2009,South Asia,4.402,7.540,0.552,51.200,0.679,0.190,0.850
2,Afghanistan,2010,South Asia,4.758,7.647,0.539,51.600,0.600,0.121,0.707
3,Afghanistan,2011,South Asia,3.832,7.620,0.521,51.920,0.496,0.162,0.731
4,Afghanistan,2012,South Asia,3.783,7.705,0.521,52.240,0.531,0.236,0.776
...,...,...,...,...,...,...,...,...,...,...
2093,Zimbabwe,2017,Sub-Saharan Africa,3.638,8.016,0.754,55.000,0.753,-0.098,0.751
2094,Zimbabwe,2018,Sub-Saharan Africa,3.616,8.049,0.775,55.600,0.763,-0.068,0.844
2095,Zimbabwe,2019,Sub-Saharan Africa,2.694,7.950,0.759,56.200,0.632,-0.064,0.831
2096,Zimbabwe,2020,Sub-Saharan Africa,3.160,7.829,0.717,56.800,0.643,-0.009,0.789


### Country Data

In [11]:
# Find the non matching country names in the country_data.csv file
country_data = pd.read_csv("Data/country_data.csv")

common_countries = happiness_df.merge(country_data, on=["Country"])
missing_countries = happiness_df[~happiness_df['Country'].isin(common_countries['Country'])]
missing_countries

,Country,Region,Year,Happiness Score,GDP per Capita,Social Support,Life Expectancy,Freedom,Generosity,Corruption
23,Taiwan Province of China,East Asia,2021,6.584,10.871,0.898,69.600,0.784,-0.070,0.721
32,Kosovo,Central and Eastern Europe,2021,6.372,9.318,0.821,63.813,0.869,0.257,0.917
64,Moldova,Commonwealth of Independent States,2021,5.766,9.454,0.857,65.699,0.822,-0.079,0.918
73,North Cyprus,Western Europe,2021,5.536,10.576,0.820,73.898,0.795,0.012,0.626
76,Hong Kong S.A.R. of China,East Asia,2021,5.477,11.000,0.836,76.820,0.717,0.067,0.403
82,Congo (Brazzaville),Sub-Saharan Africa,2021,5.342,8.117,0.636,58.221,0.695,-0.068,0.745
93,North Macedonia,Central and Eastern Europe,2021,5.101,9.693,0.805,65.474,0.751,0.038,0.905
99,Laos,Southeast Asia,2021,5.030,8.947,0.728,58.968,0.910,0.123,0.658
117,Iran,Middle East and North Africa,2021,4.721,9.584,0.710,66.300,0.608,0.218,0.714
124,Palestinian Territories,Middle East and North Africa,2021,4.517,8.485,0.826,62.250,0.653,-0.163,0.821


In [ ]:
# COUNTRY DATA

# rename the non-matching countries identified aboove 
# Rename list for country_data.csv
rename_list = [["Hong Kong", "Hong Kong S.A.R. of China"], 
               ["Taiwan", "Taiwan Province of China", "Taiwan"], 
               ["Congo", "Congo (Brazzaville)", ], 
               ["Macedonia, the former Yugoslav Republic of", "North Macedonia"],  
               ["Palestinian Territory, Occupied", "Palestinian Territories"],
               ["Lao People's Democratic Republic", "Laos"],
               ["Iran, Islamic Republic of", "Iran"],
               ["Tanzania, United Republic of", "Tanzania"],
               ["Moldova, Republic of", "Moldova"]]

# Change the country names to match happiness_df
for item in range (len(rename_list)):
    index = country_data[country_data["Country"] == rename_list[item][0]].index
    country_data.loc[index, 'Country'] = rename_list[item][1]

# Merge with happienss data
country_df = pd.merge(happiness_df[["Country", "Region", "Happiness Score"]], country_data, how='left')

# Add Alpha-2 & 3 codes to happiness df to allow easier merging later on 
happiness_df["Alpha-2 code"] = country_df["Alpha-2 code"]
happiness_df.to_csv("Data/Final Dataframes/happiness_df.csv", index=False)

# Write to csv and display
country_df.to_csv("Data/Final Dataframes/country_df.csv", index=False)
country_df


### Weather Data

In [ ]:
# Temp and weather data (1991-2016) accessed from World Bank Climate Knowledge Portal
# Availabile at: https://climateknowledgeportal.worldbank.org/download-data
# Sunlight data: World Cities Ranked by Annual Sunshine Hours. Dataset accessed from Kaggle

# TEMPERATURE DATA
# Temp data units = degrees Celsius
temp_data = pd.read_csv("Data/Temp_data.csv")

# Take the mean of each countries monthly average betweeb 1991-2016
temp_data = temp_data.groupby(['ISO3']).mean().reset_index()
# Change column name and drop unneccesary columns
temp_data = temp_data.rename(columns={"Temperature - (Celsius)":"Average Temp"})
temp_data = temp_data.drop(['Year'], axis=1)

# RAINFALL DATA
# Rainfall data units = mm
rainfall_data = pd.read_csv("Data/Rainfall_data.csv")

# Take the mean of each countries monthly average betweeb 1991-2016
rainfall_data = rainfall_data.groupby(['ISO3']).mean().reset_index()
# Change column name and drop unneccesary columns
rainfall_data = rainfall_data.rename(columns={"Rainfall - (MM)":"Average Rainfall"})
rainfall_data = rainfall_data.drop(['Year'], axis=1)

# Merge the temp and rainfall data into weather_data
weather_data = pd.merge(temp_data, rainfall_data, on="ISO3")
weather_data

# SUNLIGHT DATA
# Sunlight data units = annual sunlight hours
sun_data = pd.read_csv("Data/Sunlight_data.csv")
country_data = pd.read_csv("Data/country_data.csv")
sun_data = pd.merge(sun_data, country_data[['Country', "Alpha-3 code"]], on="Country")
sun_data = sun_data.drop(['Country'], axis=1)

# Add sunlight data to weather dataframe
weather_data = pd.merge(weather_data, sun_data, how="left", left_on="ISO3", right_on="Alpha-3 code")
weather_data = weather_data.drop(["Alpha-3 code"], axis=1)
weather_data.drop_duplicates(keep='first', inplace=True)

# Add happiness data
weather_data = pd.merge(country_df[['Country', 'Region', 'Happiness Score', 'Alpha-3 code']],
                        weather_data, how="left", right_on="ISO3", left_on="Alpha-3 code")
weather_data.drop("ISO3", axis=1, inplace=True)

# write to csv and display 
weather_data.reset_index(drop=True)
weather_data.to_csv("Data/Final Dataframes/weather_data.csv", index=False)
weather_data

### Mental Health Data

In [ ]:
# MENTAL HEALTH DATA
# Source: WHO - Depression and Other Common Mental Disorders
# Availabile at: https://www.who.int/publications/i/item/depression-global-health-estimates

# Load data from mental_health.csv and conver rates to percentage
mental_health_data = pd.read_csv("Data/mental_health.csv", encoding='latin1')
mental_health_data['Depression rate'] = mental_health_data['Depression rate'] *100
mental_health_data['Anxiety rate'] = mental_health_data['Anxiety rate'] *100

# Merge with the country data
mental_health_data = pd.merge(happiness_df[['Country', 'Region', 'Happiness Score']], mental_health_data, on="Country", how="left")

# Write to csv and display 
mental_health_data.reset_index(drop=True)
mental_health_data.to_csv("Data/Final Dataframes/mental_health_data.csv", index=False)
mental_health_data


### COVID Data

In [ ]:
# COVID DATA API REQUEST

# Load country_data.csv 
country_data = pd.read_csv("Data/country_data.csv")

# Create a dataframe to store the COVID data
covid_data = pd.DataFrame(columns={"Covid Cases", "Covid Deaths", "Covid Death Rate", "Covid Cases per Mil Pop", "Alpha-2 code"})
covid_data.insert(0, "Country", country_data["Country"])
covid_data
base_url = "http://corona-api.com/countries/"

# Use iterrows to iterate through the dataframe, adding each countires covid data
counter = 0
for index, row in country_data.iterrows():

    try:
        code = row["Alpha-2 code"]
    
        # call the COVID data API
        response = requests.get(base_url + code.lower()).json()
        
        # pull data from response
        covid_cases = response["data"]["latest_data"]["confirmed"]
        covid_deaths = response["data"]["latest_data"]["deaths"]
        covid_death_rate = response["data"]["latest_data"]["calculated"]["death_rate"]
        cases_per_mil_pop = response["data"]["latest_data"]["calculated"]["cases_per_million_population"]
    
        covid_data.loc[index, "Covid Cases"] = float(covid_cases)
        covid_data.loc[index, "Covid Deaths"] = float(covid_deaths)
        covid_data.loc[index, "Covid Cases per Mil Pop"] = float(cases_per_mil_pop)
        covid_data.loc[index, "Alpha-2 code"] = code
        
        if covid_death_rate != None:
            covid_data.loc[index, "Covid Death Rate"] = float(covid_death_rate)
        else:
            covid_data.loc[index, "Covid Death Rate"] = covid_death_rate

        counter += 1
        print(f"{round((counter/len(covid_data)*100),2)}% Done")
    
    except: 
        country = country_data.loc[index, "Country"]
        print(f"Error with country: {country}. No data found")


covid_data.to_csv("Data/covid_data.csv", index=False)
covid_data

In [ ]:
# Pull data from covid_data.csv (to save repeating API call)
data = pd.read_csv("Data/covid_data.csv")
final_covid_data = pd.merge(happiness_df[["Country", "Region", "Happiness Score", "Alpha-2 code"]], 
                            data[["Covid Cases", "Covid Deaths", "Covid Death Rate", "Covid Cases per Mil Pop", "Alpha-2 code"]], 
                            how="left", on="Alpha-2 code")

# Drop duplicates
final_covid_data.drop_duplicates(keep='last', inplace=True)

# Write to csv and display
final_covid_data.reset_index(drop=True)
final_covid_data.to_csv("Data/Final Dataframes/final_covid_data.csv", index=False)
final_covid_data

### Combined Data

In [ ]:
# Merge country data
final_df = pd.merge(happiness_df, country_df[["Alpha-2 code", "Alpha-3 code", "Numeric code", 
                                              "Latitude (average)", "Longitude (average)", "Population 2020",
                                              "Med. Age", "Urban Pop %"]], how='left', on="Alpha-2 code")


# Merge weather data
final_df = pd.merge(final_df, weather_data[['Alpha-3 code', 'Average Temp', 'Average Rainfall', 'Sunlight']], 
                    on='Alpha-3 code', how="left")

# Merge mental health data
final_df = pd.merge(final_df, mental_health_data[["Country", "Depression rate", "Anxiety rate"]], how="left", on="Country")

# Merge COVID data
final_df = pd.merge(final_df, final_covid_data[["Covid Cases", "Covid Deaths", "Covid Death Rate", "Covid Cases per Mil Pop", "Alpha-2 code"]], how="left", on="Alpha-2 code")

# # Drop duplicates, blanks
final_df.drop_duplicates(keep='last', inplace=True)
final_df.dropna(subset=["Alpha-3 code"], inplace=True)

# # Write to csv
# final_df.reset_index(drop=True)
final_df.to_csv("Data/Final Dataframes/final_df.csv", index=False)
final_df

In [ ]:
final_df.columns

### How Happy is Australia?

In [ ]:
# Reduce the happiness_time dataframe to just Australia data
australia_happiness = happiness_time_final[happiness_time_final['Country'] == 'Australia']

# Write to csv
australia_happiness.to_csv("Data/Final Dataframes/australia_happiness.csv", index=False)

# Show Australia Happiness dataframe
australia_happiness

In [ ]:
# Reduce the happiness_time dataframe to just Finland data
finland_happiness = happiness_time_final[happiness_time_final['Country'] == 'Finland']

# Write to csv
finland_happiness.to_csv("Data/Final Dataframes/finland_happiness.csv", index=False)

# Show Finland Happiness dataframe
finland_happiness

In [ ]:
# Reduce the happiness_time dataframe to just Afghanistan data
afghanistan_happiness = happiness_time_final[happiness_time_final['Country'] == 'Afghanistan']

# Write to csv
afghanistan_happiness.to_csv("Data/Final Dataframes/afghanistan_happiness.csv", index=False)

# Show Afghanistan Happiness dataframe
afghanistan_happiness

In [ ]:
# Group the average scores of final_df dataframe by Region
avg_scores_region = final_df.groupby(["Region"]).mean()
avg_scores_region = avg_scores_region.sort_values(['Happiness Score'])

# Write to csv
avg_scores_region.to_csv("Data/Final Dataframes/avg_scores_region.csv", index=False)

# Show avg_scores_region dataframe
avg_scores_region

In [ ]:
# Set variable holding happiness_data in time, per country
happiness_map_time = happiness_time_final[['Year', 'Country', 'Happiness Score']]
happiness_map_time = happiness_map_time.sort_values(['Year'])

# Write to csv
happiness_map_time.to_csv("Data/Final Dataframes/happiness_map_time.csv", index=False)

# Show happiness_map_time dataframe
happiness_map_time

In [ ]:
# Global average scores per year
avg_scores = happiness_time_final.groupby(["Year"]).mean()

# Write to csv
avg_scores.to_csv("Data/Final Dataframes/avg_scores.csv", index=False)

# Show avg_scores dataframe
avg_scores

In [ ]:
# Checking length of data for each year 
len_data_year = happiness_time_final.groupby(['Year']).count()

# Write to csv
len_data_year.to_csv("Data/Final Dataframes/len_data_year.csv", index=False)

# Show avg_scores dataframe
len_data_year